In [3]:

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm
from torchsummary import summary

In [6]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        #Fresh start
        self.conv1  = nn.Conv2d(1, 8, 3, padding=1)
        self.bn1    = nn.BatchNorm2d(8)
        self.conv2  = nn.Conv2d(8, 12, 3, padding=1)
        self.bn2    = nn.BatchNorm2d(12)
        self.pool1  = nn.MaxPool2d(2,2)

        self.conv3  = nn.Conv2d(12, 16, 3, padding=1)
        self.bn3    = nn.BatchNorm2d(16)
        self.conv4  = nn.Conv2d(16, 20, 3, padding=1)
        self.bn4    = nn.BatchNorm2d(20)
        self.pool2  = nn.MaxPool2d(2, 2)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.05)
        self.fc1 = nn.Linear(20*7*7, 10)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        

        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        

        x = x.view(-1, 20*7*7)
        x = self.fc1(x)
        
        return F.log_softmax(x)

In [9]:
use_cuda = torch.cuda.is_available()
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = Network().to(device)
#summary(model, input_size=(1, 28,28))

network = Network()
total_parameters = sum(p.numel() for p in network.parameters())
print(f"Total Parameters : {total_parameters}")
print(network)

mps
Total Parameters : 15522
Network(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(12, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(16, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (dropout): Dropout(p=0.05, inplace=False)
  (fc1): Linear(in_features=980, out_features=10, bias=True)
)


In [10]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        #  transforms.RandomApply([transforms.CenterCrop(22)], p=0.1),
                        #  transforms.Resize((28, 28)),
                         #transforms.RandomRotation((-15., 15.)),
                         transforms.RandomRotation(15),
                        # transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        #loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

model = Network().to(device)
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(0, 20):
    print(epoch+1)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

1


  0%|          | 0/469 [00:00<?, ?it/s]/var/folders/w9/1gz_7_dn4zs29tysldrxvfnh0000gn/T/ipykernel_60197/1360035597.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.0537085235118866 batch_id=468: 100%|██████████| 469/469 [00:10<00:00, 43.95it/s]  



Test set: Average loss: 0.0623, Accuracy: 9790/10000 (97.90%)

2


loss=0.06075376644730568 batch_id=468: 100%|██████████| 469/469 [00:10<00:00, 46.69it/s] 



Test set: Average loss: 0.0314, Accuracy: 9897/10000 (98.97%)

3


...


Test set: Average loss: 0.0200, Accuracy: 9943/10000 (99.43%)

19


loss=0.02650272659957409 batch_id=468: 100%|██████████| 469/469 [00:10<00:00, 46.17it/s]  



Test set: Average loss: 0.0212, Accuracy: 9938/10000 (99.38%)

20


loss=0.06205792352557182 batch_id=468: 100%|██████████| 469/469 [00:10<00:00, 45.95it/s]  



Test set: Average loss: 0.0226, Accuracy: 9928/10000 (99.28%)

